In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()

from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import log_loss
import lightgbm as lgb

In [2]:
DATA_DIR = '/kaggle/input/march-machine-learning-mania-2023'

In [3]:
MRSCResults = pd.read_csv(DATA_DIR + '/MRegularSeasonCompactResults.csv')

A_w = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_A"})
N_w = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_N"})
H_w = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_H"})
win = A_w.join(N_w, how='outer').join(H_w, how='outer').fillna(0)

H_l = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_H"})
N_l = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_N"})
A_l = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_A"})
lost = A_l.join(N_l, how='outer').join(H_l, how='outer').fillna(0)

win.index = win.index.rename(['Season', 'TeamID'])
lost.index = lost.index.rename(['Season', 'TeamID'])
wl = win.join(lost, how='outer').reset_index()
wl['win_pct_A'] = wl['win_A'] / (wl['win_A'] + wl['lost_A'])
wl['win_pct_N'] = wl['win_N'] / (wl['win_N'] + wl['lost_N'])
wl['win_pct_H'] = wl['win_H'] / (wl['win_H'] + wl['lost_H'])
wl['win_pct_All'] = (wl['win_A'] + wl['win_N'] + wl['win_H']) / \
    (wl['win_A'] + wl['win_N'] + wl['win_H'] + wl['lost_A']\
     + wl['lost_N'] + wl['lost_H'])

del A_w, N_w, H_w, H_l, N_l, A_l, win, lost
wl.head()

,Season,TeamID,win_A,win_N,win_H,lost_A,lost_N,lost_H,win_pct_A,win_pct_N,win_pct_H,win_pct_All
0,1985,1102,3.0,0.0,2.0,9.0,2.0,8.0,0.250000,0.000000,0.200000,0.208333
1,1985,1103,2.0,0.0,7.0,8.0,0.0,6.0,0.200000,NaN,0.538462,0.391304
2,1985,1104,5.0,1.0,15.0,6.0,2.0,1.0,0.454545,0.333333,0.937500,0.700000
3,1985,1106,1.0,1.0,8.0,10.0,3.0,1.0,0.090909,0.250000,0.888889,0.416667
4,1985,1108,8.0,3.0,8.0,4.0,1.0,1.0,0.666667,0.750000,0.888889,0.760000


In [4]:
MRSCResults['relScore'] = MRSCResults.WScore - MRSCResults.LScore

w_scr = MRSCResults.loc[:, ['Season', 'WTeamID', 'WScore', 'WLoc','relScore']]
w_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
l_scr = MRSCResults.loc[:, ['Season', 'LTeamID', 'LScore', 'WLoc','relScore']]
l_scr['WLoc'] = l_scr.WLoc.apply(lambda x: 'H' if x == 'A' else 'A' \
                                 if x == 'H' else 'N')
l_scr['relScore'] = -1 * l_scr.relScore 
l_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
wl_scr = pd.concat([w_scr,l_scr])

A_scr = wl_scr[wl_scr.Loc == 'A'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_A", "relScore": "relScore_A"})
N_scr = wl_scr[wl_scr.Loc == 'N'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_N", "relScore": "relScore_N"})
H_scr = wl_scr[wl_scr.Loc == 'H'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_H", "relScore": "relScore_H"})
All_scr = wl_scr.groupby(['Season','TeamID'])['Score','relScore']\
    .mean().rename(columns={"Score": "Score_All", "relScore": "relScore_All"})
scr = A_scr.join(N_scr, how='outer').join(H_scr, how='outer')\
    .join(All_scr, how='outer').fillna(0).reset_index()

del w_scr, l_scr, wl_scr, A_scr, H_scr, N_scr, All_scr
scr.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,Season,TeamID,Score_A,relScore_A,Score_N,relScore_N,Score_H,relScore_H,Score_All,relScore_All
0,1985,1102,62.833333,-8.166667,55.50,-12.500000,64.900000,-1.600000,63.083333,-5.791667
1,1985,1103,60.800000,-8.300000,0.00,0.000000,61.230769,1.000000,61.043478,-3.043478
2,1985,1104,65.181818,4.000000,53.00,-0.333333,73.687500,11.937500,68.500000,7.800000
3,1985,1106,72.272727,-7.272727,64.50,-14.000000,74.000000,5.000000,71.625000,-3.791667
4,1985,1108,78.583333,1.583333,77.75,5.250000,91.222222,17.666667,83.000000,7.960000


In [5]:
MRSDetailedResults = pd.read_csv(DATA_DIR + '/MRegularSeasonDetailedResults.csv')

w = MRSDetailedResults.loc[:, ['Season', 'WTeamID', 'WFGM','WFGA','WFGM3'
                               ,'WFGA3','WFTM','WFTA','WOR','WDR','WAst',
                               'WTO','WStl','WBlk','WPF']]
w.columns = ['Season', 'TeamID', 'FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR',
             'Ast','TO','Stl','Blk','PF']
l = MRSDetailedResults.loc[:, ['Season', 'LTeamID', 'LFGM','LFGA','LFGM3',
                               'LFGA3','LFTM','LFTA','LOR','LDR','LAst',
                               'LTO','LStl','LBlk','LPF']]
l.columns = ['Season', 'TeamID', 'FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR',
             'Ast','TO','Stl','Blk','PF']

detail = pd.concat([w,l])
detail['goal_rate'] = detail.FGM / detail.FGA 
detail['3p_goal_rate'] = detail.FGM3 / detail.FGA3  
detail['ft_goal_rate'] = detail.FTM  / detail.FTA  

dt = detail.groupby(['Season','TeamID'])['FGM','FGA','FGM3','FGA3','FTM','FTA',
                                         'OR','DR','Ast','TO','Stl','Blk','PF',
                                          'goal_rate', '3p_goal_rate',
                                         'ft_goal_rate']\
                                        .mean().fillna(0).reset_index()

del w, l, detail
dt.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,Season,TeamID,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,goal_rate,3p_goal_rate,ft_goal_rate
0,2003,1102,19.142857,39.785714,7.821429,20.821429,11.142857,17.107143,4.178571,16.821429,13.000000,11.428571,5.964286,1.785714,18.750000,0.486149,0.367637,0.642402
1,2003,1103,27.148148,55.851852,5.444444,16.074074,19.037037,25.851852,9.777778,19.925926,15.222222,12.629630,7.259259,2.333333,19.851852,0.487294,0.331990,0.735271
2,2003,1104,24.035714,57.178571,6.357143,19.857143,14.857143,20.928571,13.571429,23.928571,12.107143,13.285714,6.607143,3.785714,18.035714,0.419676,0.325442,0.705168
3,2003,1105,24.384615,61.615385,7.576923,20.769231,15.423077,21.846154,13.500000,23.115385,14.538462,18.653846,9.307692,2.076923,20.230769,0.396204,0.359630,0.709598
4,2003,1106,23.428571,55.285714,6.107143,17.642857,10.642857,16.464286,12.285714,23.857143,11.678571,17.035714,8.357143,3.142857,18.178571,0.425530,0.350196,0.623158


In [6]:
MMOrdinals = pd.read_csv('/kaggle/input/march-machine-learning-mania-2023/MMasseyOrdinals_thru_Season2023_Day128.csv')

MOR_127_128 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 127) \
                 | (MMOrdinals.RankingDayNum == 128))]\
                [['Season','TeamID','OrdinalRank']]
MOR_50_51 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 50) \
                 | (MMOrdinals.RankingDayNum == 51))]\
                [['Season','TeamID','OrdinalRank']]
MOR_15_16 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 15) \
                 | (MMOrdinals.RankingDayNum == 16))]\
                [['Season','TeamID','OrdinalRank']]

MOR_127_128 = MOR_127_128.rename(columns={'OrdinalRank':'OrdinalRank_127_128'})
MOR_50_51 = MOR_50_51.rename(columns={'OrdinalRank':'OrdinalRank_50_51'})
MOR_15_16 = MOR_15_16.rename(columns={'OrdinalRank':'OrdinalRank_15_16'})

MOR = MOR_127_128.merge(MOR_50_51, how='left', on=['Season','TeamID'])\
        .merge(MOR_15_16, how='left', on=['Season','TeamID'])\

## normalizing Rank values by its season maxium as it varies by seasons
MOR_max = MOR.groupby('Season')['OrdinalRank_127_128', 'OrdinalRank_50_51',
                                'OrdinalRank_15_16'].max().reset_index()
MOR_max.columns = ['Season', 'maxRank_127_128','maxRank_50_51', 'maxRank_15_16']

MOR_tmp = MMOrdinals[(MMOrdinals.SystemName == 'MOR') \
                     & (MMOrdinals.RankingDayNum < 133)]
MOR_stats = MOR_tmp.groupby(['Season','TeamID'])['OrdinalRank']\
            .agg(['max','min','std','mean']).reset_index()
MOR_stats.columns = ['Season','TeamID','RankMax','RankMin','RankStd','RankMean']

MOR = MOR.merge(MOR_max, how='left', on='Season')\
        .merge(MOR_stats, how='left', on=['Season','TeamID'])
MOR['OrdinalRank_127_128'] = MOR['OrdinalRank_127_128'] / MOR['maxRank_127_128']
MOR['OrdinalRank_50_51'] = MOR['OrdinalRank_50_51'] / MOR['maxRank_50_51']
MOR['OrdinalRank_15_16'] = MOR['OrdinalRank_15_16'] / MOR['maxRank_15_16']

MOR['RankTrans_50_51_to_127_128'] = MOR['OrdinalRank_127_128'] - MOR['OrdinalRank_50_51']
MOR['RankTrans_15_16_to_127_128'] = MOR['OrdinalRank_127_128'] - MOR['OrdinalRank_15_16']


del MOR_127_128, MOR_50_51, MOR_15_16, MOR_max, MOR_tmp, MOR_stats
MOR.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,Season,TeamID,OrdinalRank_127_128,OrdinalRank_50_51,OrdinalRank_15_16,maxRank_127_128,maxRank_50_51,maxRank_15_16,RankMax,RankMin,RankStd,RankMean,RankTrans_50_51_to_127_128,RankTrans_15_16_to_127_128
0,2003,1102,0.418960,0.388379,NaN,327,327.0,NaN,146,106,12.670160,127.928571,0.030581,NaN
1,2003,1103,0.431193,0.501529,NaN,327,327.0,NaN,184,129,15.144723,156.142857,-0.070336,NaN
2,2003,1104,0.058104,0.045872,NaN,327,327.0,NaN,41,13,8.482691,22.571429,0.012232,NaN
3,2003,1105,0.948012,0.850153,NaN,327,327.0,NaN,314,262,17.052762,296.785714,0.097859,NaN
4,2003,1106,0.896024,0.807339,NaN,327,327.0,NaN,293,242,11.841675,268.928571,0.088685,NaN


In [7]:
POM_127_128 = MMOrdinals[(MMOrdinals.SystemName == 'POM') & \
                ((MMOrdinals.RankingDayNum == 127) \
                 | (MMOrdinals.RankingDayNum == 128))]\
                [['Season','TeamID','OrdinalRank']]
POM_50_51 = MMOrdinals[(MMOrdinals.SystemName == 'POM') & \
                ((MMOrdinals.RankingDayNum == 50) \
                 | (MMOrdinals.RankingDayNum == 51))]\
                [['Season','TeamID','OrdinalRank']]
POM_15_16 = MMOrdinals[(MMOrdinals.SystemName == 'POM') & \
                ((MMOrdinals.RankingDayNum == 15) \
                 | (MMOrdinals.RankingDayNum == 16))]\
                [['Season','TeamID','OrdinalRank']]

POM_127_128 = POM_127_128.rename(columns={'OrdinalRank':'OrdinalRank_127_128_POM'})
POM_50_51 = POM_50_51.rename(columns={'OrdinalRank':'OrdinalRank_50_51_POM'})
POM_15_16 = POM_15_16.rename(columns={'OrdinalRank':'OrdinalRank_15_16_POM'})

POM = POM_127_128.merge(POM_50_51, how='left', on=['Season','TeamID'])\
        .merge(POM_15_16, how='left', on=['Season','TeamID'])\

## normalizing Rank values by its season maxium as it varies by seasons
POM_max = POM.groupby('Season')['OrdinalRank_127_128_POM', 'OrdinalRank_50_51_POM',
                                'OrdinalRank_15_16_POM'].max().reset_index()
POM_max.columns = ['Season', 'maxRank_127_128_POM','maxRank_50_51_POM', 'maxRank_15_16_POM']

POM_tmp = MMOrdinals[(MMOrdinals.SystemName == 'POM') \
                     & (MMOrdinals.RankingDayNum < 133)]
POM_stats = POM_tmp.groupby(['Season','TeamID'])['OrdinalRank']\
            .agg(['max','min','std','mean']).reset_index()
POM_stats.columns = ['Season','TeamID','RankMax','RankMin','RankStd','RankMean']

POM = POM.merge(POM_max, how='left', on='Season')\
        .merge(POM_stats, how='left', on=['Season','TeamID'])
POM['OrdinalRank_127_128_POM'] = POM['OrdinalRank_127_128_POM'] / POM['maxRank_127_128_POM']
POM['OrdinalRank_50_51_POM'] = POM['OrdinalRank_50_51_POM'] / POM['maxRank_50_51_POM']
POM['OrdinalRank_15_16_POM'] = POM['OrdinalRank_15_16_POM'] / POM['maxRank_15_16_POM']

POM['RankTrans_50_51_to_127_128_POM'] = POM['OrdinalRank_127_128_POM'] - POM['OrdinalRank_50_51_POM']
POM['RankTrans_15_16_to_127_128_POM'] = POM['OrdinalRank_127_128_POM'] - POM['OrdinalRank_15_16_POM']

POM.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,Season,TeamID,OrdinalRank_127_128_POM,OrdinalRank_50_51_POM,OrdinalRank_15_16_POM,maxRank_127_128_POM,maxRank_50_51_POM,maxRank_15_16_POM,RankMax,RankMin,RankStd,RankMean,RankTrans_50_51_to_127_128_POM,RankTrans_15_16_to_127_128_POM
0,2003,1102,0.495413,0.385321,NaN,327,327.0,NaN,167,84,24.750624,138.857143,0.110092,NaN
1,2003,1103,0.483180,0.522936,NaN,327,327.0,NaN,202,158,13.206933,180.500000,-0.039755,NaN
2,2003,1104,0.076453,0.055046,NaN,327,327.0,NaN,43,14,9.474373,26.071429,0.021407,NaN
3,2003,1105,0.941896,0.941896,NaN,327,327.0,NaN,319,305,4.242641,312.000000,0.000000,NaN
4,2003,1106,0.782875,0.792049,NaN,327,327.0,NaN,259,170,23.427231,235.714286,-0.009174,NaN


In [8]:
wl_1 = wl.loc[:,['Season','TeamID','win_pct_A','win_pct_N',
                 'win_pct_H','win_pct_All']]
wl_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                else str(col) for col in wl_1.columns ]

wl_2 = wl.loc[:,['Season','TeamID','win_pct_A','win_pct_N',
                 'win_pct_H','win_pct_All']]
wl_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                else str(col) for col in wl_2.columns ]

scr_1 = scr.copy()
scr_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                 else str(col) for col in scr_1.columns ]

scr_2 = scr.copy()
scr_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                 else str(col) for col in scr_2.columns ]

dt_1 = dt.copy()
dt_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                else str(col) for col in dt_1.columns ]

dt_2 = dt.copy()
dt_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                else str(col) for col in dt_2.columns ]

MOR_1 = MOR.copy()
MOR_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                 else str(col) for col in MOR_1.columns ]

MOR_2 = MOR.copy()
MOR_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                 else str(col) for col in MOR_2.columns ]

POM_1 = POM.copy()
POM_1.columns = [str(col) + '_1_POM' if col not in ['Season','TeamID'] \
                 else str(col) for col in POM_1.columns ]

POM_2 = POM.copy()
POM_2.columns = [str(col) + '_2_POM' if col not in ['Season','TeamID'] \
                 else str(col) for col in POM_2.columns ]

In [9]:
TCResults = pd.read_csv(DATA_DIR + '/MNCAATourneyCompactResults.csv')

tourney1 = TCResults.loc[:, ['Season','WTeamID','LTeamID']]
tourney1.columns = ['Season','TeamID1','TeamID2']
tourney1['result'] = 1

tourney2 = TCResults.loc[:, ['Season','LTeamID','WTeamID']]
tourney2.columns = ['Season','TeamID1','TeamID2']
tourney2['result'] = 0

tourney = pd.concat([tourney1, tourney2])
del tourney1, tourney2

In [10]:
def merge_data(df):

    df = df.merge(wl_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(wl_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)


    df = df.merge(scr_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(scr_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)

    df = df.merge(dt_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(dt_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)

    df = df.merge(POM_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(POM_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)

    df = df.fillna(-1)
    
    for col in df.columns:
        if (df[col] == np.inf).any() or (df[col] == -np.inf).any():
            df[col][(df[col] == np.inf) | (df[col] == -np.inf)] = -1
    
    return df

tourney = merge_data(tourney)
tourney = tourney.loc[tourney.Season >= 2003,:].reset_index(drop=True)

In [11]:
tourney

,Season,TeamID1,TeamID2,result,win_pct_A_1,win_pct_N_1,win_pct_H_1,win_pct_All_1,win_pct_A_2,win_pct_N_2,...,OrdinalRank_15_16_POM_2_POM,maxRank_127_128_POM_2_POM,maxRank_50_51_POM_2_POM,maxRank_15_16_POM_2_POM,RankMax_2_POM,RankMin_2_POM,RankStd_2_POM,RankMean_2_POM,RankTrans_50_51_to_127_128_POM_2_POM,RankTrans_15_16_to_127_128_POM_2_POM
0,2003,1421,1411,1,0.125000,1.000000,0.833333,0.448276,0.333333,0.750000,...,-1.000000,327.0,327.0,-1.0,298.0,218.0,24.181093,259.571429,0.018349,-1.000000
1,2003,1112,1436,1,0.916667,0.000000,0.933333,0.892857,0.500000,0.600000,...,-1.000000,327.0,327.0,-1.0,197.0,133.0,16.728603,166.000000,-0.003058,-1.000000
2,2003,1113,1272,1,0.400000,0.500000,0.800000,0.620690,0.636364,1.000000,...,-1.000000,327.0,327.0,-1.0,79.0,22.0,18.460680,48.214286,-0.073394,-1.000000
3,2003,1141,1166,1,0.642857,1.000000,0.916667,0.793103,0.636364,1.000000,...,-1.000000,327.0,327.0,-1.0,34.0,2.0,11.347653,14.000000,0.097859,-1.000000
4,2003,1143,1301,1,0.636364,0.250000,0.928571,0.724138,0.300000,0.500000,...,-1.000000,327.0,327.0,-1.0,77.0,11.0,16.601503,51.071429,0.039755,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,2022,1274,1242,0,0.833333,0.500000,0.666667,0.696970,0.600000,0.857143,...,0.008380,358.0,358.0,358.0,10.0,2.0,2.411675,5.764706,0.013966,0.016760
2492,2022,1389,1314,0,0.500000,1.000000,0.666667,0.620690,0.727273,0.200000,...,0.139665,358.0,358.0,358.0,53.0,24.0,8.487014,37.823529,-0.013966,-0.050279
2493,2022,1437,1242,0,0.615385,0.857143,0.923077,0.787879,0.600000,0.857143,...,0.008380,358.0,358.0,358.0,10.0,2.0,2.411675,5.764706,0.013966,0.016760
2494,2022,1181,1314,0,0.818182,0.800000,0.833333,0.823529,0.727273,0.200000,...,0.139665,358.0,358.0,358.0,53.0,24.0,8.487014,37.823529,-0.013966,-0.050279


In [12]:
MSampleSubmission = pd.read_csv(DATA_DIR + '/SampleSubmission2023.csv')

test1 = MSampleSubmission.copy()
test1['Season'] = test1.ID.apply(lambda x: int(x[0:4]))
test1['TeamID1'] = test1.ID.apply(lambda x: int(x[5:9]))
test1['TeamID2'] = test1.ID.apply(lambda x: int(x[10:14]))

test2 = MSampleSubmission.copy()
test2['Season'] = test2.ID.apply(lambda x: int(x[0:4]))
test2['TeamID1'] = test2.ID.apply(lambda x: int(x[10:14]))
test2['TeamID2'] = test2.ID.apply(lambda x: int(x[5:9]))

test = pd.concat([test1,test2]).drop(['Pred'], axis=1)
test = merge_data(test)

## Remove Women Id

In [13]:
wTeam = pd.read_csv("/kaggle/input/march-machine-learning-mania-2023/WRegularSeasonDetailedResults.csv")
wTeam_Id = wTeam["WTeamID"]
wTeam_Id = set(wTeam_Id.tolist())
test = test[~test["ID"].str.split("_").apply(lambda x: any(int(i) in wTeam_Id for i in x[1:]))]

In [14]:
test

,ID,Season,TeamID1,TeamID2,win_pct_A_1,win_pct_N_1,win_pct_H_1,win_pct_All_1,win_pct_A_2,win_pct_N_2,...,OrdinalRank_15_16_POM_2_POM,maxRank_127_128_POM_2_POM,maxRank_50_51_POM_2_POM,maxRank_15_16_POM_2_POM,RankMax_2_POM,RankMin_2_POM,RankStd_2_POM,RankMean_2_POM,RankTrans_50_51_to_127_128_POM_2_POM,RankTrans_15_16_to_127_128_POM_2_POM
0,2023_1101_1102,2023,1101,1102,0.333333,0.00,0.500000,0.346154,0.333333,0.000000,...,0.647383,363.0,363.0,363.0,260.0,141.0,36.848058,176.823529,-0.046832,-0.245179
1,2023_1101_1103,2023,1101,1103,0.333333,0.00,0.500000,0.346154,0.454545,0.333333,...,0.344353,363.0,363.0,363.0,127.0,95.0,11.020035,115.235294,-0.046832,-0.041322
2,2023_1101_1104,2023,1101,1104,0.333333,0.00,0.500000,0.346154,0.750000,0.714286,...,0.041322,363.0,363.0,363.0,15.0,2.0,4.633637,7.705882,-0.024793,-0.033058
3,2023_1101_1105,2023,1101,1105,0.333333,0.00,0.500000,0.346154,0.384615,0.333333,...,0.925620,363.0,363.0,363.0,342.0,302.0,11.667892,327.529412,-0.066116,-0.085399
4,2023_1101_1106,2023,1101,1106,0.333333,0.00,0.500000,0.346154,0.100000,1.000000,...,0.958678,363.0,363.0,363.0,352.0,335.0,4.727112,345.705882,-0.008264,0.002755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196381,2023_1474_1476,2023,1476,1474,0.333333,0.50,0.600000,0.433333,0.411765,1.000000,...,0.617080,363.0,363.0,363.0,224.0,183.0,12.265147,204.941176,0.060606,-0.041322
196382,2023_1474_1477,2023,1477,1474,0.368421,0.25,0.444444,0.375000,0.411765,1.000000,...,0.617080,363.0,363.0,363.0,224.0,183.0,12.265147,204.941176,0.060606,-0.041322
196383,2023_1475_1476,2023,1476,1475,0.333333,0.50,0.600000,0.433333,0.312500,0.500000,...,0.550964,363.0,363.0,363.0,273.0,193.0,25.387294,241.470588,0.112948,0.201102
196384,2023_1475_1477,2023,1477,1475,0.368421,0.25,0.444444,0.375000,0.312500,0.500000,...,0.550964,363.0,363.0,363.0,273.0,193.0,25.387294,241.470588,0.112948,0.201102


In [15]:
X = tourney.drop(['Season','TeamID1','TeamID2','result'], axis=1)
y = tourney["result"]
s = tourney["Season"]

X_test = test.drop(['ID', 'Season','TeamID1','TeamID2'], axis=1)

In [16]:
X_test

,win_pct_A_1,win_pct_N_1,win_pct_H_1,win_pct_All_1,win_pct_A_2,win_pct_N_2,win_pct_H_2,win_pct_All_2,Score_A_1,relScore_A_1,...,OrdinalRank_15_16_POM_2_POM,maxRank_127_128_POM_2_POM,maxRank_50_51_POM_2_POM,maxRank_15_16_POM_2_POM,RankMax_2_POM,RankMin_2_POM,RankStd_2_POM,RankMean_2_POM,RankTrans_50_51_to_127_128_POM_2_POM,RankTrans_15_16_to_127_128_POM_2_POM
0,0.333333,0.00,0.500000,0.346154,0.333333,0.000000,0.526316,0.437500,71.500000,-6.666667,...,0.647383,363.0,363.0,363.0,260.0,141.0,36.848058,176.823529,-0.046832,-0.245179
1,0.333333,0.00,0.500000,0.346154,0.454545,0.333333,0.928571,0.645161,71.500000,-6.666667,...,0.344353,363.0,363.0,363.0,127.0,95.0,11.020035,115.235294,-0.046832,-0.041322
2,0.333333,0.00,0.500000,0.346154,0.750000,0.714286,1.000000,0.852941,71.500000,-6.666667,...,0.041322,363.0,363.0,363.0,15.0,2.0,4.633637,7.705882,-0.024793,-0.033058
3,0.333333,0.00,0.500000,0.346154,0.384615,0.333333,0.428571,0.400000,71.500000,-6.666667,...,0.925620,363.0,363.0,363.0,342.0,302.0,11.667892,327.529412,-0.066116,-0.085399
4,0.333333,0.00,0.500000,0.346154,0.100000,1.000000,0.444444,0.233333,71.500000,-6.666667,...,0.958678,363.0,363.0,363.0,352.0,335.0,4.727112,345.705882,-0.008264,0.002755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196381,0.333333,0.50,0.600000,0.433333,0.411765,1.000000,0.500000,0.500000,64.444444,-7.333333,...,0.617080,363.0,363.0,363.0,224.0,183.0,12.265147,204.941176,0.060606,-0.041322
196382,0.368421,0.25,0.444444,0.375000,0.411765,1.000000,0.500000,0.500000,68.684211,-5.947368,...,0.617080,363.0,363.0,363.0,224.0,183.0,12.265147,204.941176,0.060606,-0.041322
196383,0.333333,0.50,0.600000,0.433333,0.312500,0.500000,0.636364,0.448276,64.444444,-7.333333,...,0.550964,363.0,363.0,363.0,273.0,193.0,25.387294,241.470588,0.112948,0.201102
196384,0.368421,0.25,0.444444,0.375000,0.312500,0.500000,0.636364,0.448276,68.684211,-5.947368,...,0.550964,363.0,363.0,363.0,273.0,193.0,25.387294,241.470588,0.112948,0.201102


In [17]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import KFold, GroupKFold
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm
import glob
import os
import gc
import xgboost as xgb

train = tourney
test = test

xgb_params= {
        "objective": "binary:logistic",
        "max_depth": 10,
        "learning_rate": 0.1,
        "colsample_bytree": 0.8,
        "subsample": 0.8,
        "min_child_weight": 20,
        "n_jobs": 2,
        "seed": 2023,

    }

y = train["result"]
s = train["Season"]
X = train.drop(['Season','TeamID1','TeamID2','result'], axis=1)

X_test = test.drop(['ID', 'Season','TeamID1','TeamID2'], axis=1)

train_oof = np.zeros((X.shape[0],))
test_preds = 0
train_oof.shape

NUM_FOLDS = 5
kf = GroupKFold(n_splits=NUM_FOLDS)
max_iter = 550

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(X, y, s))):
        train_df, val_df = X.iloc[train_ind], X.iloc[val_ind]
        train_target, val_target = y.iloc[train_ind], y.iloc[val_ind]
        train_df_xgb = xgb.DMatrix(train_df, label=train_target)
        val_df_xgb = xgb.DMatrix(val_df, label=val_target)
        
        model = HistGradientBoostingClassifier(max_iter=max_iter, validation_fraction=None, learning_rate=0.01, max_depth=2, min_samples_leaf=32)
        model1 = RandomForestClassifier()
        model2 = LogisticRegression(C=1)
#         model3 = SVC(probability=True)
        model4 = xgb.train(xgb_params, train_df_xgb)

        model =  model.fit(train_df, train_target)
        model1 =  model1.fit(train_df, train_target)
        model2 =  model2.fit(train_df, train_target)

#         temp_oof = model4.predict(val_df_xgb)
#         temp_test = model4.predict(xgb.DMatrix(X_test))
        
        temp_oof = (
                    model1.predict_proba(val_df)[:,1] + \
                    model4.predict(val_df_xgb)) / 2
        temp_test = (

                     model1.predict_proba(X_test)[:,1] + \
                     model4.predict(xgb.DMatrix(X_test))) / 2

        train_oof[val_ind] = temp_oof
        test_preds += temp_test/NUM_FOLDS
        
        print(log_loss(val_target, temp_oof))
        
print('CV', log_loss(y, train_oof))        
np.save('train_oof', train_oof)
np.save('test_preds', test_preds)

test = test


MSampleSubmission = pd.read_csv('/kaggle/input/march-machine-learning-mania-2023/SampleSubmission2023.csv')

idx = test_preds.shape[0] // 2
test_preds[idx:] = 1 - test_preds[idx:]

pred = pd.concat([test.ID, pd.Series(test_preds)], axis=1).groupby('ID')[0]\
        .mean().reset_index().rename(columns={0:'Pred'})
sub3 = MSampleSubmission.drop(['Pred'],axis=1).merge(pred, on='ID')
pred_3 = sub3['Pred']

0it [00:00, ?it/s]

0.6000768069855426
0.5645899903800345
0.5762089001410042
0.5696584426110911
0.5504465034476677
CV 0.5708110309142752


In [18]:
sub3

,ID,Pred
0,2023_1101_1102,0.484329
1,2023_1101_1103,0.499447
2,2023_1101_1104,0.362583
3,2023_1101_1105,0.540044
4,2023_1101_1106,0.481036
...,...,...
65698,2023_1474_1476,0.543722
65699,2023_1474_1477,0.538532
65700,2023_1475_1476,0.508824
65701,2023_1475_1477,0.506266


In [19]:
xxx = pd.read_csv("/kaggle/input/mmlm2023-predictions/logistic_baseline_mmlm2023_men_submission.csv")

In [20]:
sub3.merge(xxx, on="ID")

,ID,Pred,pred
0,2023_1104_1112,0.548968,0.581444
1,2023_1104_1113,0.723231,0.851591
2,2023_1104_1116,0.668816,0.760738
3,2023_1104_1120,0.702522,0.784381
4,2023_1104_1124,0.638260,0.662747
...,...,...,...
2273,2023_1436_1452,0.302946,0.318648
2274,2023_1436_1462,0.277486,0.207031
2275,2023_1438_1452,0.500194,0.622280
2276,2023_1438_1462,0.487327,0.479488


In [21]:
sub3.to_csv("xgb_mmlm2023_v2.csv", index=False)

In [22]:
sub3.head()

,ID,Pred
0,2023_1101_1102,0.484329
1,2023_1101_1103,0.499447
2,2023_1101_1104,0.362583
3,2023_1101_1105,0.540044
4,2023_1101_1106,0.481036
